# Import Library 

In [ ]:
import pandas
import configparser
import psycopg2

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['db']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [ ]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

# Create House Table 

In [ ]:
table_sql = """
            CREATE TABLE IF NOT EXISTS gp23.house
            (

                price integer,
                bed integer,
                bath integer,
                area integer,
                address VARCHAR(200),
                PRIMARY KEY(address)
            );

            """

In [ ]:
conn.rollback()


In [ ]:

cur.execute(table_sql)
conn.commit()

# Define the Table 

In [ ]:

url = 'https://www.trulia.com/VA/Ashburn/20147/'

In [ ]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
print(html_data.decode('utf-8'))

In [ ]:

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
print (soup)

In [ ]:
for li_class in soup.find_all('li', class_ = 'Grid__CellBox-sc-144isrp-0 SearchResultsList__WideCell-b7y9ki-2 jiZmPM'):
    try:
        for price_div in li_class.find_all('div',{'data-testid':'property-price'}):
            price =int(price_div.text.replace('$','').replace(",",""))
        for bed_div in li_class.find_all('div', {'data-testid':'property-beds'}):
            bed= int(bed_div.text.replace('bd','').replace(",",""))
        for bath_div in li_class.find_all('div',{'data-testid':'property-baths'}):
            bath =int(bath_div.text.replace('ba','').replace(",",""))
        for area_div in li_class.find_all('div',{'data-testid':'property-floorSpace'}):
            area=int(area_div.text.split('sqft')[0].replace(",",""))
        for address_div in li_class.find_all('div',{'data-testid':'property-address'}):
            address =address_div.text

        try:
            sql_insert = """
                        insert into gp23.house(price,bed,bath,area,address)
                        values('{}','{}','{}','{}','{}')            
                        """.format(price,bed,bath,area,address)

            cur.execute(sql_insert)
            conn.commit()
        except:
            conn.rollback()
    except:
        pass

# Query the Table

In [ ]:
df = pandas.read_sql_query('select * from gp23.house ', conn)
df[:]

# Basic Stat

In [ ]:
df.describe()

# Price Distribution 

In [ ]:
df['price'].hist()

# Bed vs Bath 

In [ ]:
df.plot.scatter(x='bed',y='bath')